In [1]:
import pandas as pd

In [2]:
telemetry = pd.read_csv('/content/PdM_telemetry.csv')
errors = pd.read_csv('/content/PdM_errors.csv')
maint = pd.read_csv('/content/PdM_maint.csv')
failures = pd.read_csv('/content/PdM_failures.csv')
machines = pd.read_csv('/content/PdM_machines.csv')

In [3]:
telemetry['datetime'] = pd.to_datetime(
    telemetry['datetime'], format="%Y-%m-%d %H:%M:%S")

telemetry.describe()

,datetime,machineID,volt,rotate,pressure,vibration
count,876100,876100.000000,876100.000000,876100.000000,876100.000000,876100.000000
mean,2015-07-02 18:00:00,50.500000,170.777736,446.605119,100.858668,40.385007
min,2015-01-01 06:00:00,1.000000,97.333604,138.432075,51.237106,14.877054
25%,2015-04-02 12:00:00,25.750000,160.304927,412.305714,93.498181,36.777299
50%,2015-07-02 18:00:00,50.500000,170.607338,447.558150,100.425559,40.237247
75%,2015-10-02 00:00:00,75.250000,181.004493,482.176600,107.555231,43.784938
max,2016-01-01 06:00:00,100.000000,255.124717,695.020984,185.951998,76.791072
std,NaN,28.866087,15.509114,52.673886,11.048679,5.370361


In [4]:
errors['datetime'] = pd.to_datetime(
    errors['datetime'], format="%Y-%m-%d %H:%M:%S")
errors['errorID'] = errors['errorID'].astype('object')
errors.head()

,datetime,machineID,errorID
0,2015-01-03 07:00:00,1,error1
1,2015-01-03 20:00:00,1,error3
2,2015-01-04 06:00:00,1,error5
3,2015-01-10 15:00:00,1,error4
4,2015-01-22 10:00:00,1,error4


In [5]:
maint['datetime'] = pd.to_datetime(
    maint['datetime'], format="%Y-%m-%d %H:%M:%S")
maint['comp'] = maint['comp'].astype('object')
maint.head()

,datetime,machineID,comp
0,2014-06-01 06:00:00,1,comp2
1,2014-07-16 06:00:00,1,comp4
2,2014-07-31 06:00:00,1,comp3
3,2014-12-13 06:00:00,1,comp1
4,2015-01-05 06:00:00,1,comp4


In [6]:
machines['model'] = machines['model'].astype('object')
machines.head()

,machineID,model,age
0,1,model3,18
1,2,model4,7
2,3,model3,8
3,4,model3,7
4,5,model3,2


In [7]:
failures['datetime'] = pd.to_datetime(
    failures['datetime'], format="%Y-%m-%d %H:%M:%S")
failures['failure'] = failures['failure'].astype('object')
failures.head()

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4


In [8]:
error_count = pd.get_dummies(errors.set_index('datetime')).reset_index()
error_count.columns = ['datetime', 'machineID',
                       'error1', 'error2', 'error3', 'error4', 'error5']
error_count[['error1', 'error2', 'error3', 'error4', 'error5']] = error_count[['error1', 'error2', 'error3', 'error4', 'error5']].astype(int)
error_count.head()

,datetime,machineID,error1,error2,error3,error4,error5
0,2015-01-03 07:00:00,1,1,0,0,0,0
1,2015-01-03 20:00:00,1,0,0,1,0,0
2,2015-01-04 06:00:00,1,0,0,0,0,1
3,2015-01-10 15:00:00,1,0,0,0,1,0
4,2015-01-22 10:00:00,1,0,0,0,1,0


In [9]:
error_count = telemetry[['datetime', 'machineID']].merge(
    error_count, on=['machineID', 'datetime'], how='left').fillna(0)
error_count.describe()

,datetime,machineID,error1,error2,error3,error4,error5
count,876403,876403.000000,876403.000000,876403.000000,876403.000000,876403.00000,876403.000000
mean,2015-07-02 18:00:05.931517952,50.499881,0.001152,0.001127,0.000956,0.00083,0.000406
min,2015-01-01 06:00:00,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,2015-04-02 12:00:00,25.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,2015-07-02 18:00:00,50.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,2015-10-02 00:00:00,76.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,2016-01-01 06:00:00,100.000000,1.000000,1.000000,1.000000,1.00000,1.000000
std,NaN,28.866464,0.033928,0.033557,0.030907,0.02879,0.020150


In [10]:
import numpy as np

comp_rep = pd.get_dummies(maint.set_index('datetime')).reset_index()
comp_rep.columns = ['datetime', 'machineID',
                    'comp1', 'comp2', 'comp3', 'comp4']

comp_rep[['comp1', 'comp2', 'comp3', 'comp4']] = comp_rep[['comp1', 'comp2', 'comp3', 'comp4']].astype(int)

comp_rep = telemetry[['datetime', 'machineID']].merge(comp_rep,
                                                      on=['datetime',
                                                          'machineID'],
                                                      how='outer').fillna(0).sort_values(by=['machineID', 'datetime'])
comp_rep.head()

,datetime,machineID,comp1,comp2,comp3,comp4
876823,2014-06-01 06:00:00,1,0.0,1.0,0.0,0.0
876824,2014-07-16 06:00:00,1,0.0,0.0,0.0,1.0
876825,2014-07-31 06:00:00,1,0.0,0.0,1.0,0.0
876826,2014-12-13 06:00:00,1,1.0,0.0,0.0,0.0
0,2015-01-01 06:00:00,1,0.0,0.0,0.0,0.0


In [11]:
final_feat = telemetry.merge(
    error_count, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(
    comp_rep, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(machines, on=['machineID'], how='left')

final_feat.head()

,datetime,machineID,volt,rotate,pressure,vibration,error1,error2,error3,error4,error5,comp1,comp2,comp3,comp4,model,age
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18


In [12]:
labeled_features = final_feat.merge(
    failures, on=['datetime', 'machineID'], how='left')
labeled_features = labeled_features.fillna(
    method='bfill', limit=7)
labeled_features = labeled_features.fillna('none')
labeled_features.head()

<ipython-input-12-2f4b1a168d96>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  labeled_features = labeled_features.fillna(


,datetime,machineID,volt,rotate,pressure,vibration,error1,error2,error3,error4,error5,comp1,comp2,comp3,comp4,model,age,failure
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,none
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,none
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,none
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,none
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,none


In [13]:
labeled_features.loc[labeled_features['failure'] == 'comp4'][:16]

,datetime,machineID,volt,rotate,pressure,vibration,error1,error2,error3,error4,error5,comp1,comp2,comp3,comp4,model,age,failure
89,2015-01-04 23:00:00,1,167.023756,471.194987,89.474271,62.464103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,comp4
90,2015-01-05 00:00:00,1,161.928938,376.719605,89.969588,58.087488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,comp4
91,2015-01-05 01:00:00,1,127.163620,430.475185,95.667178,46.845600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,comp4
92,2015-01-05 02:00:00,1,158.155297,429.346217,94.892589,58.225151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,comp4
93,2015-01-05 03:00:00,1,177.317220,461.801246,103.056344,49.894647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,comp4
94,2015-01-05 04:00:00,1,202.520488,387.005318,94.686208,45.202347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,comp4
95,2015-01-05 05:00:00,1,177.510419,469.787301,100.498426,59.577251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,comp4
96,2015-01-05 06:00:00,1,179.303153,499.777962,111.833028,52.383097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,model3,18,comp4
97,2015-01-05 06:00:00,1,179.303153,499.777962,111.833028,52.383097,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,model3,18,comp4
4054,2015-06-18 23:00:00,1,167.160880,435.781251,94.930887,50.986459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,model3,18,comp4


In [14]:
threshold_dates = [
    [pd.to_datetime('2015-07-31 01:00:00'), pd.to_datetime('2015-08-01 01:00:00')],
    [pd.to_datetime('2015-08-31 01:00:00'), pd.to_datetime('2015-09-01 01:00:00')],
    [pd.to_datetime('2015-09-30 01:00:00'), pd.to_datetime('2015-10-01 01:00:00')]
]

train_dfs = []
test_dfs = []

for last_train_date, first_test_date in threshold_dates:
    print('Training on %s to %s' % (last_train_date, first_test_date))

    # Creating the training set
    train_data = labeled_features[labeled_features['datetime'] < last_train_date]
    train_y = train_data['failure']
    train_X = pd.get_dummies(train_data.drop(['datetime', 'machineID', 'failure'], axis=1))
    train_df = pd.concat([train_X, train_y], axis=1)
    train_dfs.append(train_df)

    # Creating the testing set
    test_data = labeled_features[(labeled_features['datetime'] >= first_test_date) &
                                 (labeled_features['datetime'] < first_test_date + pd.Timedelta(days=1))]
    test_y = test_data['failure']
    test_X = pd.get_dummies(test_data.drop(['datetime', 'machineID', 'failure'], axis=1))
    test_df = pd.concat([test_X, test_y], axis=1)
    test_dfs.append(test_df)

Training on 2015-07-31 01:00:00 to 2015-08-01 01:00:00
Training on 2015-08-31 01:00:00 to 2015-09-01 01:00:00
Training on 2015-09-30 01:00:00 to 2015-10-01 01:00:00


In [15]:
train_dfs[0].head()

,volt,rotate,pressure,vibration,error1,error2,error3,error4,error5,comp1,comp2,comp3,comp4,age,model_model1,model_model2,model_model3,model_model4,failure
0,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,False,False,True,False,none
1,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,False,False,True,False,none
2,170.989902,527.349825,75.237905,34.178847,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,False,False,True,False,none
3,162.462833,346.149335,109.248561,41.122144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,False,False,True,False,none
4,157.610021,435.376873,111.886648,25.990511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,False,False,True,False,none


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
periods = []
accuracies = []

for i, (train_df, test_df) in enumerate(zip(train_dfs, test_dfs)):
    train_X = train_df.drop('failure', axis=1)
    train_y = train_df['failure']

    test_X = test_df.drop('failure', axis=1)
    test_y = test_df['failure']

    model = RandomForestClassifier(n_estimators=100, max_depth=5)
    model.fit(train_X, train_y)

    y_pred = model.predict(test_X)

    accuracy = accuracy_score(test_y, y_pred)

    periods.append(f"Period {i+1}")
    accuracies.append(accuracy)

metrics_df = pd.DataFrame({
    'Period': periods,
    'Accuracy': accuracies
})

In [17]:
metrics_df.to_csv('metrics.csv', index=False)